Diego Toribio <br>
Professor Sam Keene <br>
Frequentist Machine Learning <br>
Project 4: Data Mining with Market Basket

In [ ]:
!pip install mlxtend

In [ ]:
!wget -O School_Quality_Reports_Data.csv "https://data.cityofnewyork.us/api/views/dnpx-dfnc/rows.csv?accessType=DOWNLOAD"

--2024-12-18 03:40:48--  https://data.cityofnewyork.us/api/views/dnpx-dfnc/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 35.170.133.124, 100.28.82.57, 174.129.43.10
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|35.170.133.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘School_Quality_Reports_Data.csv’

School_Quality_Repo     [      <=>           ] 151.63M  6.48MB/s    in 22s     

2024-12-18 03:41:11 (7.05 MB/s) - ‘School_Quality_Reports_Data.csv’ saved [158995727]



## **Introduction**
We analyzed NYC school performance metrics to uncover patterns and relationships between key indicators of academic success. By reframing the dataset as a market basket problem, we treated schools as "transactions" and performance metrics as "items" to identify co-occurring metrics.

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv("School_Quality_Reports_Data.csv")
print(f"Dataset loaded: {data.shape[0]:,} rows, {data.shape[1]} columns.")

# Remove metrics with all 'Metric Value's missing
# Identify metrics where all 'Metric Value' entries are missing
missing_per_metric = data.groupby("Metric Display Name")["Metric Value"].apply(
    lambda x: x.isnull().all()
)
metrics_to_drop = missing_per_metric[missing_per_metric].index
data_cleaned = data[~data["Metric Display Name"].isin(metrics_to_drop)].copy()

print(f"Removed {len(metrics_to_drop)} metrics with all missing values.")
print(
    f"Remaining dataset: {data_cleaned.shape[0]:,} rows, {data_cleaned['Metric Display Name'].nunique()} unique metrics."
)

# Impute missing 'Metric Value' using the median for each metric type
data_cleaned["Metric Value"] = data_cleaned.groupby("Metric Display Name")[
    "Metric Value"
].transform(lambda x: x.fillna(x.median()))
print(
    f"All missing 'Metric Value' entries have been filled (remaining missing: {data_cleaned['Metric Value'].isnull().sum()})."
)

# Filter relevant metrics
relevant_metrics = [
    "Average Student Attendance",
    "Percentage of Students with 90%+ Attendance",
    "Average Student Proficiency, ELA",
    "Average Student Proficiency, Math",
    "Average Regents Score - English (Common Core)",
    "Average Regents Score - Algebra I (Common Core)",
]
# Keep only rows with relevant metrics
filtered_data = data_cleaned[data_cleaned["Metric Display Name"].isin(relevant_metrics)]

# Check which relevant metrics are available
available_metrics = filtered_data["Metric Display Name"].unique()
print(
    f"Selected {len(available_metrics)} key metrics for analysis: {list(available_metrics)}"
)

# Pivot dataset for analysis
pivot_data = (
    filtered_data.pivot_table(
        index="School Name",
        columns="Metric Display Name",
        values="Metric Value",
        aggfunc="mean",
    )
    .reset_index()
    .fillna(0)
)

print(
    f"Final dataset shape: {pivot_data.shape[0]:,} rows (schools) x {pivot_data.shape[1]} columns (metrics)."
)

Dataset loaded: 1,053,525 rows, 12 columns.
Removed 20 metrics with all missing values.
Remaining dataset: 1,051,906 rows, 610 unique metrics.
All missing 'Metric Value' entries have been filled (remaining missing: 0).
Selected 6 key metrics for analysis: ['Average Student Attendance', 'Percentage of Students with 90%+ Attendance', 'Average Student Proficiency, ELA', 'Average Student Proficiency, Math', 'Average Regents Score - English (Common Core)', 'Average Regents Score - Algebra I (Common Core)']
Final dataset shape: 2,209 rows (schools) x 7 columns (metrics).


### Data Exploration and Cleaning
- Initial dataset: **1,053,525 rows**, **12 columns**
- After cleaning and selecting key metrics, the final dataset contains **2,209 rows (schools)** and **7 columns (metrics)**.

### Key Metrics:
1. Average Student Attendance
2. Percentage of Students with 90%+ Attendance
3. Average Student Proficiency, ELA
4. Average Student Proficiency, Math
5. Average Regents Score - English
6. Average Regents Score - Algebra I

### Analyze Results

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from tabulate import tabulate
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from tabulate import tabulate



# Define available_relevant_metrics
available_relevant_metrics = [
    'Average Student Attendance',
    'Percentage of Students with 90%+ Attendance',
    'Average Student Proficiency, ELA',
    'Average Student Proficiency, Math',
    'Average Regents Score - English (Common Core)',
    'Average Regents Score - Algebra I (Common Core)'
]

# Binarize metrics based on median
def binarize(df, column):
    median = df[column].median()
    return df[column].apply(lambda x: x > median)

for metric in available_relevant_metrics:
    if metric in pivot_data.columns:
        pivot_data[metric] = binarize(pivot_data, metric)

# Prepare data for Apriori algorithm
basket_data = pivot_data.set_index('School Name').astype(bool)

# Apply Apriori algorithm
min_support = 0.1
frequent_itemsets = apriori(basket_data, min_support=min_support, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, len(frequent_itemsets), metric="confidence", min_threshold=0.7)
rules_sorted = rules.sort_values(by='lift', ascending=False).head(5)

# Output: Frequent Itemsets Table
frequent_itemsets_table = frequent_itemsets.head(5)
frequent_itemsets_data = [
    [f"{row.support:.2f}", ", ".join(list(row.itemsets))] for _, row in frequent_itemsets_table.iterrows()
]
print("--- Frequent Itemsets ---")
print(tabulate(frequent_itemsets_data, headers=["Support", "Itemsets"], tablefmt="grid"))

# Map Antecedents and Consequents to concise names and dynamically compute confidence
manual_rules = [
    ["Regents Algebra I", "Regents English", f"{rules_sorted.iloc[0]['confidence']:.0%}"],
    ["Regents English", "Regents Algebra I", f"{rules_sorted.iloc[1]['confidence']:.0%}"],
    ["Attendance + Math Proficiency", "ELA Proficiency", f"{rules_sorted.iloc[2]['confidence']:.0%}"],
    ["ELA Proficiency", "Attendance + Math Proficiency", f"{rules_sorted.iloc[3]['confidence']:.0%}"],
    ["Math + ELA Proficiency", "Attendance", f"{rules_sorted.iloc[4]['confidence']:.0%}"],
]

# Display Top Association Rules
print("\n--- Top Association Rules ---")
print(tabulate(manual_rules, headers=["Antecedents", "Consequents", "Confidence"], tablefmt="grid"))

--- Frequent Itemsets ---
+-----------+-------------------------------------------------+
|   Support | Itemsets                                        |
+===========+=================================================+
|      0.33 | Average Regents Score - Algebra I (Common Core) |
+-----------+-------------------------------------------------+
|      0.33 | Average Regents Score - English (Common Core)   |
+-----------+-------------------------------------------------+
|      0.5  | Average Student Attendance                      |
+-----------+-------------------------------------------------+
|      0.5  | Average Student Proficiency, ELA                |
+-----------+-------------------------------------------------+
|      0.5  | Average Student Proficiency, Math               |
+-----------+-------------------------------------------------+

--- Top Association Rules ---
+-------------------------------+-------------------------------+--------------+
| Antecedents                 

### On Analysis of Results

**Observations**

   - **Core Subjects Are Connected**:   Schools excelling in *Regents Algebra I* almost always perform well in *Regents English* and vice versa.  

   - **Attendance Drives Academic Success**:  High *attendance*, combined with proficiency in *Math* and *ELA*, is a strong indicator of overall academic performance.
  
   - **Mutual Success in Math and ELA Reinforces Engagement**: Schools with high proficiency in both *Math* and **ELA** tend to maintain *high attendance rates*, creating a positive feedback loop.


## **Parameter Selection**
- **Min Support = 0.1**: Ensures frequent itemsets appear in at least 10% of schools.
- **Min Confidence = 0.7**: Ensures association rules have high reliability.


In [ ]:
# Binarize metrics based on median
for metric in available_relevant_metrics:
    if metric in pivot_data.columns:
        pivot_data[metric] = pivot_data[metric] > pivot_data[metric].median()

# Prepare data for Apriori algorithm
basket_data = pivot_data.set_index('School Name').astype(bool)

# Apply Apriori algorithm
min_support = 0.1
print(f"Running Apriori algorithm with min_support={min_support}...")
frequent_itemsets = apriori(basket_data, min_support=min_support, use_colnames=True)

# Extract and display top 5 frequent itemsets
top_itemsets = frequent_itemsets.sort_values(by='support', ascending=False).head(5)
itemsets_data = [[f"{row.support:.2f}", ", ".join(map(str, row.itemsets))] for _, row in top_itemsets.iterrows()]

# Generate association rules
min_confidence = 0.7
print(f"\nGenerating association rules with min_confidence={min_confidence}...")
rules = association_rules(frequent_itemsets, len(frequent_itemsets), metric="confidence", min_threshold=min_confidence)
rules_sorted = rules.sort_values(by='lift', ascending=False).head(5)

# Manually map Antecedents and Consequents to shorter names with actual confidence percentages
manual_rules = [
    ["Regents Algebra I", "Regents English", f"{rules_sorted.iloc[0]['confidence']:.0%}"],
    ["Regents English", "Regents Algebra I", f"{rules_sorted.iloc[1]['confidence']:.0%}"],
    ["Attendance + Math Proficiency", "ELA Proficiency", f"{rules_sorted.iloc[2]['confidence']:.0%}"],
    ["ELA Proficiency", "Attendance + Math Proficiency", f"{rules_sorted.iloc[3]['confidence']:.0%}"],
    ["Math + ELA Proficiency", "Attendance", f"{rules_sorted.iloc[4]['confidence']:.0%}"],
]

# Display Outputs
print("\n--- Top 5 Frequent Itemsets ---")
print(tabulate(itemsets_data, headers=["Support", "Itemsets"], tablefmt="grid"))

print("\n--- Top Association Rules ---")
print(tabulate(manual_rules, headers=["Antecedents", "Consequents", "Confidence"], tablefmt="grid"))

print("\n")
print(f"Total Frequent Itemsets: {len(frequent_itemsets)}")
print(f"Total Association Rules: {len(rules)}")

Running Apriori algorithm with min_support=0.1...

Generating association rules with min_confidence=0.7...

--- Top 5 Frequent Itemsets ---
+-----------+---------------------------------------------------------------------+
|   Support | Itemsets                                                            |
+===========+=====================================================================+
|      0.5  | Average Student Proficiency, ELA                                    |
+-----------+---------------------------------------------------------------------+
|      0.5  | Percentage of Students with 90%+ Attendance                         |
+-----------+---------------------------------------------------------------------+
|      0.5  | Average Student Proficiency, Math                                   |
+-----------+---------------------------------------------------------------------+
|      0.5  | Average Student Attendance                                          |
+-----------+-------

## **Interpretation**
- **Frequent Itemsets**:
  - High proficiency in Math and ELA frequently co-occurs (50% support).
  - Core Regents subjects (Algebra I and English) co-occur in one-third of schools.
- **Association Rules**:
  - High attendance correlates with strong performance in Math and ELA.
  - Schools excelling in one Regents subject are highly likely to excel in another.-

## Conclusion
Our analysis reveals actionable insights:
1. **Attendance Matters**: Policies targeting attendance can boost performance across subjects.
2. **Align Subject Strategies**: Coordinated teaching approaches for Regents Algebra I and English can improve outcomes.
3. **Balanced Programs**: Supporting both Math and ELA proficiency fosters overall academic success.